In [4]:
import requests
import os

Day = 16

# get file from website using private session key stored in enviromental variables
r = requests.get(
            f'https://adventofcode.com/2023/day/'+str(Day)+'/input',
            cookies={'session': os.getenv('AdventSessionKey')}
)

# read r.text
grid = r.text.strip().split('\n')


In [1]:
def follow_beam(grid, beams):
    directions = {'right': (0, 1), 'down': (1, 0), 'left': (0, -1), 'up': (-1, 0)}
    mirrors = {'/': {'up': 'right', 'right': 'up', 'down': 'left', 'left': 'down'},
               '\\': {'up': 'left', 'right': 'down', 'down': 'right', 'left': 'up'}}
    splitters = {'|': {'up': 'up', 'right': ['up', 'down'], 'down': 'down', 'left': ['up', 'down']},
                 '-': {'up': ['left', 'right'], 'right': 'right', 'down': ['left', 'right'], 'left': 'left'}}
    
    # all the beams ever created, so we don't create duplicates
    master_catalog = set()
    
    output_grid = [['.' for _ in range(len(grid[0]))] for _ in range(len(grid))]

    while beams:

        # Store a list of new beams during each "beams" for loop
        new_beams = []

        for beam in beams:

            # start next beam
            position, direction = beam['position'], beam['direction']
            End_Beam = False

            # Debug
            # print(f"Beam Position: {position}, Beam Direction: {direction}")  

            # while on the grid and still going
            while 0 <= position[0] < len(grid) and 0 <= position[1] < len(grid[0]) and not End_Beam:
                
                # energize the output grid
                output_grid[position[0]][position[1]] = '#'
                
                # update the direction(s) based on the symbol
                if grid[position[0]][position[1]] in mirrors:
                    direction = mirrors[grid[position[0]][position[1]]][direction]
                elif grid[position[0]][position[1]] in splitters:
                    direction = splitters[grid[position[0]][position[1]]][direction]
                
                # If the beam split, make two new beams and end the current one
                if isinstance(direction, list):
                    
                    # end the current beam
                    End_Beam = True
                    
                    # Attempt to make two new beams
                    for dir in direction:
                        # Move one step in given direction before storing
                        new_position = [position[0] + directions[dir][0], position[1] + directions[dir][1]]
                        
                        # Make sure the beam is on the grid beofre storing
                        if 0 <= new_position[0] < len(grid) and 0 <= new_position[1] < len(grid[0]):
                            new_beam = {'position': new_position, 'direction': dir}
                            # also, make sure you didn't already store the same beam
                            if str(new_beam) not in master_catalog:
                                # add to  set of beams for the next while loop
                                new_beams.append(new_beam)
                                # add to the master catalog
                                master_catalog.add(str(new_beam))
                
                else:
                    # Beam was not split; move beam, update the beam position and direction
                    position[0] += directions[direction][0]
                    position[1] += directions[direction][1]
    
        # for loop is done, give the new set of beams to the while loop
        beams = new_beams

    return output_grid


In [2]:
# # Test 
# grid = [
#     list('.|...\\....'),
#     list('|.-.\\.....'),
#     list('.....|-...'),
#     list('........|.'),
#     list('..........'),
#     list('.........\\'),
#     list('..../.\\\\..'),
#     list('.-.-/..|..'),
#     list('.|....-|\\.'),
#     list('..//.|....')
# ]

# Part 1
beams = [{'position': [0, 0], 'direction': 'right'}]
Energized_grid = follow_beam(grid, beams)

hash_count = 0

for row in Energized_grid:
    hash_count += row.count('#')   

print(f"\nPart 1: Number of energized tiles: {hash_count}")

# 8034



Part 1: Number of energized tiles: 46


In [5]:
# Part 2 

def test_all_positions(grid):
    max_hash_count = 0
    directions = ['right', 'left', 'down', 'up']
    edges = [range(len(grid)), range(len(grid[0])-1, -1, -1), range(len(grid[0])), range(len(grid)-1, -1, -1)]
    for direction, edge in zip(directions, edges):
        for i in edge:
            if direction == 'right':
                position = [i, 0]
            elif direction == 'left':
                position = [i, len(grid[0])-1]
            elif direction == 'down':
                position = [0, i]
            elif direction == 'up':
                position = [len(grid)-1, i]
            beams = [{'position': position, 'direction': direction}]
            Energized_grid = follow_beam(grid, beams)
            hash_count = sum(row.count('#') for row in Energized_grid)
            if hash_count > max_hash_count:
                max_hash_count = hash_count
    return max_hash_count

print(f"Largest hash count: {test_all_positions(grid)}")

# 8225

Largest hash count: 8225
